In [1]:
import pandas as pd
root = 'data/'
lang = 'java'
categories = 1
if lang == 'java':
    categories = 5
    
#model_num = 5

query_source = pd.read_pickle(root+lang+'/query_source.pkl')


In [2]:
import torch
from model import BatchProgramCC
from gensim.models.word2vec import Word2Vec

def get_device():
    if torch.cuda.is_available():
        if torch.cuda.get_device_name(0) == 'GeForce GT 730':
            device = 'cpu'
        else:
            device = 'cuda'
    else:
        device = 'cpu'
    return torch.device(device)

device = get_device()

word2vec = Word2Vec.load(root+lang+"/train/embedding/node_w2v_128").wv
MAX_TOKENS = word2vec.vectors.shape[0]
EMBEDDING_DIM = word2vec.vectors.shape[1]

HIDDEN_DIM = 100
ENCODE_DIM = 128
LABELS = 1
EPOCHS = 10
BATCH_SIZE = 1

type_list = [1, 2, 3, 4, 5]
models = []
for t in type_list:
    model = BatchProgramCC(EMBEDDING_DIM,
                           HIDDEN_DIM,
                           MAX_TOKENS+1,
                           ENCODE_DIM,
                           LABELS,
                           BATCH_SIZE,
                           device
                           )

    model.to(device)
    model.load_state_dict(torch.load('code_clone_detection_java_model_train_on_30_percent_label_' + str(t) + '.pt'))
    model.eval()
    models.append(model)

C:\Users\A1\anaconda3\envs\pytorch\lib\site-packages\torch\cuda\__init__.py:102: UserWarning: 
    Found GPU0 GeForce GT 730 which is of cuda capability 3.0.
    PyTorch no longer supports this GPU because it is too old.
    The minimum cuda capability that we support is 3.5.
    
  warnings.warn(old_gpu_warn % (d, name, major, capability[1]))
C:\Users\A1\anaconda3\envs\pytorch\lib\site-packages\torch\cuda\__init__.py:125: UserWarning: 
GeForce GT 730 with CUDA capability sm_30 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_61 sm_70 sm_75 compute_37.
If you want to use the GeForce GT 730 GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))


In [3]:
class vector_calculation():
    def __init__(self, model):
        self.model = model
    def __call__(self, x):
        with torch.no_grad():
            vector = self.model.encode([x]).reshape(-1)
        return vector.cpu().numpy()
    
def get_batch(dataset, idx, bs):
    tmp = dataset.iloc[idx: idx+bs]
    x = []
    for _, item in tmp.iterrows():
        x.append(item['block'])
    return x

In [4]:
for t in type_list:
    query_source['vector-'+str(t)] = query_source['block'].apply(vector_calculation(models[t-1]))

In [ ]:
type_1_vector = []
i = 0
while i < len(query_source):
    blocks = get_batch(query_source, i, BATCH_SIZE)
    with torch.no_grad():
        model.batch_size = len(blocks)
        model.hidden = model.init_hidden()
        vectors = model.encode(blocks)
    type_1_vector.extend(vectors)
type_1_vector = [v.cpu().numpy() for v in type_1_vector]
query_source['type-2 vector fast'] = type_1_vector

In [5]:
query_source.to_pickle(root+lang+'/query_source.pkl')